In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

def upscale_image(image, scale_percent=200):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_LANCZOS4)

def apply_unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=0.5, threshold=0):
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

def remove_red_lines_and_enhance(input_path, output_path, upscale_factor=200):
    if not os.path.exists(input_path):
        print(f"File not found: {input_path}")
        return
    
    img_colored = cv2.imread(input_path, cv2.IMREAD_COLOR)
    
    if img_colored is None:
        print(f"Failed to load image from: {input_path}")
        return
    
    img_colored_upscaled = upscale_image(img_colored, upscale_factor)
    img_hsv = cv2.cvtColor(img_colored_upscaled, cv2.COLOR_BGR2HSV)
    img_ycrcb = cv2.cvtColor(img_colored_upscaled, cv2.COLOR_BGR2YCrCb)

    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 50, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)
    
    lower_red_ycrcb = np.array([0, 150, 0])
    upper_red_ycrcb = np.array([255, 255, 120])
    mask3 = cv2.inRange(img_ycrcb, lower_red_ycrcb, upper_red_ycrcb)

    red_mask = mask1 | mask2 | mask3

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    refined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    refined_mask = cv2.dilate(refined_mask, kernel, iterations=2)

    inpainted_image = img_colored_upscaled.copy()
    for _ in range(3):
        inpainted_image = cv2.inpaint(inpainted_image, refined_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

    for i in range(1, 3):
        inpainted_image = cv2.inpaint(inpainted_image, refined_mask, inpaintRadius=i, flags=cv2.INPAINT_NS)

    denoised_image = cv2.fastNlMeansDenoisingColored(inpainted_image, None, 5, 5, 7, 21)
    sharpened_image = apply_unsharp_mask(denoised_image, amount=0.3)

    lab = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced_lab = cv2.merge((cl, a, b))
    enhanced_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

    alpha = 0.8
    color_preserved = cv2.addWeighted(enhanced_image, alpha, img_colored_upscaled, 1 - alpha, 0)

    final_hsv = cv2.cvtColor(color_preserved, cv2.COLOR_BGR2HSV)
    final_mask = cv2.inRange(final_hsv, lower_red1, upper_red1) | cv2.inRange(final_hsv, lower_red2, upper_red2)
    final_mask = cv2.dilate(final_mask, kernel, iterations=1)
    color_preserved = cv2.inpaint(color_preserved, final_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

    result_img = Image.fromarray(cv2.cvtColor(color_preserved, cv2.COLOR_BGR2RGB))
    result_img.save(output_path, format="PNG", quality=100, optimize=True)

    print(f"Enhanced image with improved red line removal saved to: {output_path}")

input_path = r"C:\Users\User\Downloads\DOS_2.png"
output_path = r'C:\Users\User\Downloads\output_image_enhanced_red_removed.png'
remove_red_lines_and_enhance(input_path, output_path)
